# Étude de fraude
### Gerdy Jérôme

### Import des librairies nécessaires

In [2]:
import seaborn as sns; sns.set()
import pandas as pd

### Import des dataset Fraud_Data et IpAddress_to_Country

In [4]:
df_fraud = pd.read_csv('csv_files/Fraud_Data.csv')
df_ip = pd.read_csv('csv_files/IpAddress_to_Country.csv')

In [5]:
df_fraud.shape, df_ip.shape

((151112, 11), (138846, 3))

In [8]:
df_fraud.info()

&lt;class &#39;pandas.core.frame.DataFrame&#39;&gt;
RangeIndex: 151112 entries, 0 to 151111
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_id         151112 non-null  int64  
 1   signup_time     151112 non-null  object 
 2   purchase_time   151112 non-null  object 
 3   purchase_value  151112 non-null  int64  
 4   device_id       151112 non-null  object 
 5   source          151112 non-null  object 
 6   browser         151112 non-null  object 
 7   sex             151112 non-null  object 
 8   age             151112 non-null  int64  
 9   ip_address      151112 non-null  float64
 10  class           151112 non-null  int64  
dtypes: float64(1), int64(4), object(6)
memory usage: 12.7+ MB


In [7]:
df_ip.info()

&lt;class &#39;pandas.core.frame.DataFrame&#39;&gt;
RangeIndex: 138846 entries, 0 to 138845
Data columns (total 3 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   lower_bound_ip_address  138846 non-null  float64
 1   upper_bound_ip_address  138846 non-null  int64  
 2   country                 138846 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 3.2+ MB


In [12]:
df_ip.head()

,lower_bound_ip_address,upper_bound_ip_address,country
0,16777216.0,16777471,Australia
1,16777472.0,16777727,China
2,16777728.0,16778239,China
3,16778240.0,16779263,Australia
4,16779264.0,16781311,China


In [42]:
df_fraud.head(105)

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0
...,...,...,...,...,...,...,...,...,...,...,...
100,65674,2015-06-02 22:31:55,2015-08-23 22:49:17,20,HCKNCXWZGEMBA,Ads,IE,M,19,1.276170e+09,0
101,17259,2015-05-30 16:06:31,2015-09-03 20:48:03,38,YFDFRIQZFYEDP,SEO,Safari,M,34,9.433497e+08,0
102,259616,2015-02-16 03:40:02,2015-04-05 01:06:51,26,FIGVUJINBWPHC,SEO,Chrome,M,23,4.060442e+09,0
103,257432,2015-03-24 09:08:01,2015-04-06 02:30:40,41,HYRIZVROAHIAM,Ads,Safari,F,60,1.391044e+09,0


In [121]:
arr = []
for i, ip in df_ip.iterrows():
    if df_fraud["ip_address"].between(ip.lower_bound_ip_address, ip.upper_bound_ip_address, inclusive = True).any(axis=0) == True:
      print(df_fraud.iloc[i])

_time             2015-08-20 06:57:02
purchase_value                             36
device_id                       UCQSBRGYCQFKO
source                                    SEO
browser                                Chrome
sex                                         M
age                                        29
ip_address                        3.62819e+09
class                                       0
country                                Canada
lower_bound_ip_address            3.62818e+09
upper_bound_ip_address             3628187647
Name: 3445, dtype: object
user_id                                127468
signup_time               2015-01-31 21:30:52
purchase_time             2015-05-27 21:10:26
purchase_value                             23
device_id                       QBPTCKMQTZGKT
source                                    SEO
browser                                Chrome
sex                                         M
age                                        45
ip_address      

KeyboardInterrupt: 